In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup 
import re

from datetime import date
import sys

In [2]:
'''Testing cell to test for the function I am building below'''

team_id = '3'
start_date='2021-01-14'
end_date='2021-03-19'

url_2 = 'https://statsapi.web.nhl.com/api/v1/schedule?teamId={}&startDate={}&endDate={}'.format(
            team_id, start_date, end_date)

r_2 = requests.get(url_2, timeout=300)

r_3_link = r_2.json()['dates'][2]['games'][0]['link']

base_link = 'https://statsapi.web.nhl.com'

r_3 = requests.get(base_link+r_3_link)

In [3]:
def get_game_urls(team='New York Rangers', start_date='2021-01-14', end_date=date.today().strftime('%y-%m-%d')):
    # Get team ID for team of your choice
    base_link = 'https://statsapi.web.nhl.com'
    secondary_link = '/api/v1/teams'
    r = requests.get(base_link+secondary_link)
    json_data = r.json()
    for i in range(len(json_data['teams'])):
        if team.lower() in json_data['teams'][i]['name'].lower():
            team_id = json_data['teams'][i]['link'].split('/')[-1]
    
    # Getting the schedule of games for the specified team through the specified date range
    schedule = 'https://statsapi.web.nhl.com/api/v1/schedule?teamId={}&startDate={}&endDate={}'.format(
                team_id, start_date, end_date)
    schedule_r = requests.get(schedule, timeout=400)
    json_schedule = schedule_r.json()
    
    shot_type = []
    shot_description = []
    secondary_shot_type = []
    game_date = []
    opponent = []
    player_name = []
    player_id = []
    period = []
    period_time = []
    shot_x_coordinate = []
    shot_y_coordinate = []
    home_game = []
    head_coach = []
    
    for ixx in range(len(json_schedule['dates'])):
        game_link = json_schedule['dates'][ixx]['games'][0]['link']
        game_page = requests.get(base_link+game_link, timeout=300)
        game_page_json = game_page.json()
        
        all_plays = game_page_json['liveData']['plays']['allPlays']
        
        if game_page_json['gameData']['teams']['away']['id'] == int(team_id):
            opponent_team = game_page_json['gameData']['teams']['home']['name']
            home_or_away = 0
            game_coach = game_page_json['liveData']['boxscore']['teams']['away']['coaches']
            
        elif game_page_json['gameData']['teams']['home']['id'] == int(team_id):
            opponent_team = game_page_json['gameData']['teams']['away']['name']
            home_or_away = 1
            game_coach = game_page_json['liveData']['boxscore']['teams']['home']['coaches']
        
    
        
        for ix in range(len(all_plays)):
            single_play = all_plays[ix]
            if ('shot' in single_play['result']['event'].lower()) or \
               ('goal' in single_play['result']['event'].lower()):
                if single_play['team']['id'] == int(team_id):
                    if 'blocked' not in single_play['result']['event'].lower():
                        shot_type.append(single_play['result']['event'])
                        shot_description.append(single_play['result']['description'])
                        try:
                            secondary_shot_type.append(single_play['result']['secondaryType'])
                        except KeyError:
                            secondary_shot_type.append(np.nan)
                        game_date.append(all_plays[0]['about']['dateTime'][:10])
                        opponent.append(opponent_team)
                        player_name.append(single_play['players'][0]['player']['fullName'])
                        player_id.append(single_play['players'][0]['player']['id'])
                        period.append(single_play['about']['period'])
                        period_time.append(single_play['about']['periodTime'])
                        shot_x_coordinate.append(single_play['coordinates']['x'])
                        shot_y_coordinate.append(single_play['coordinates']['y'])
                        home_game.append(home_or_away)
                        head_coach.append(game_coach)
                    elif 'blocked' in single_play['result']['event'].lower():
                        pass
                if 'blocked' in single_play['result']['event'].lower():
                    if single_play['team']['id'] != int(team_id):
                        shot_type.append(single_play['result']['event'])
                        shot_description.append(single_play['result']['description'])
                        try:
                            secondary_shot_type.append(single_play['result']['secondaryType'])
                        except KeyError:
                            secondary_shot_type.append(np.nan)
                        game_date.append(all_plays[0]['about']['dateTime'][:10])
                        opponent.append(single_play['team']['name'])
                        player_name.append(single_play['players'][1]['player']['fullName'])
                        player_id.append(single_play['players'][1]['player']['id'])
                        period.append(single_play['about']['period'])
                        period_time.append(single_play['about']['periodTime'])
                        shot_x_coordinate.append(single_play['coordinates']['x'])
                        shot_y_coordinate.append(single_play['coordinates']['y'])
                        home_game.append(home_or_away)
                        head_coach.append(game_coach)
    df = pd.DataFrame(np.column_stack([shot_type, shot_description, secondary_shot_type, game_date, opponent, 
                                       player_name, player_id, period, period_time, 
                                       shot_x_coordinate, shot_y_coordinate, home_game, head_coach]),
                    columns=['shot_type', 'shot_description', 'secondary_shot_type', 'game_date', 
                             'opponent', 'player_name',
                        'player_id', 'period', 'period_time', 'shot_x_coordinate', 'shot_y_coordinate',
                            'home_game', 'head_coach'])
    df['head_coach'] = df.head_coach.apply(lambda x: x['person']['fullName'])
    
    return df

In [4]:
shot_data = get_game_urls()

In [5]:
shot_data

,shot_type,shot_description,secondary_shot_type,game_date,opponent,player_name,player_id,period,period_time,shot_x_coordinate,shot_y_coordinate,home_game,head_coach
0,Shot,Mika Zibanejad Wrist Shot saved by Semyon Varl...,Wrist Shot,2021-01-14,New York Islanders,Mika Zibanejad,8476459,1,00:21,2,32,1,David Quinn
1,Blocked Shot,Ryan Lindgren shot blocked shot by Matt Martin,nan,2021-01-14,New York Islanders,Ryan Lindgren,8479324,1,00:32,-72,10,1,David Quinn
2,Missed Shot,Artemi Panarin Wide of Net,nan,2021-01-14,New York Islanders,Artemi Panarin,8478550,1,01:03,-86,-11,1,David Quinn
3,Shot,Filip Chytil Wrist Shot saved by Semyon Varlamov,Wrist Shot,2021-01-14,New York Islanders,Filip Chytil,8480078,1,04:07,-84,6,1,David Quinn
4,Shot,Mika Zibanejad Slap Shot saved by Semyon Varlamov,Slap Shot,2021-01-14,New York Islanders,Mika Zibanejad,8476459,1,05:30,-70,17,1,David Quinn
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2008,Shot,Phillip Di Giuseppe Wrist Shot saved by Casey ...,Wrist Shot,2021-04-06,Pittsburgh Penguins,Phillip Di Giuseppe,8476858,3,12:07,-73,7,1,David Quinn
2009,Missed Shot,Brendan Smith Wide of Net Casey DeSmith,nan,2021-04-06,Pittsburgh Penguins,Brendan Smith,8474090,3,12:24,-85,12,1,David Quinn
2010,Blocked Shot,Phillip Di Giuseppe shot blocked shot by Cody ...,nan,2021-04-06,Pittsburgh Penguins,Phillip Di Giuseppe,8476858,3,16:10,-63,0,1,David Quinn
2011,Blocked Shot,Kaapo Kakko shot blocked shot by Frederick Gau...,nan,2021-04-06,Pittsburgh Penguins,Kaapo Kakko,8481554,3,16:36,-70,1,1,David Quinn


In [8]:
shot_data.to_csv('shot_data_01-2020--04-2021.csv')